In [1]:
import importlib
import model.trainer
import torch
import data.VCTK
importlib.reload(model.trainer)
importlib.reload(data.VCTK)

<module 'data.VCTK' from '/home/azureuser/dom/Project/data/VCTK.py'>

In [2]:
trainer = model.trainer.Trainer(
    device=torch.device("cuda"),
    checkpoint=30000,
    load_from_checkpoint=True
)

Using default model
Using default vocoder
Loading models from checkpoint 30000
Models loaded


In [3]:
dataset = model.trainer.get_dataset()

In [5]:
i=0
while True:
    total_loss, spectro_loss, vocoder_loss, (spectro_pred, wav_pred, wav_pred_temp) = trainer.train_step(dataset[0])
    i+=1
    print(
        i,
        f"total_loss={total_loss.item():.5f}",
        f"spectro_loss={spectro_loss.item():.5f}",
        f"vocoder_loss={vocoder_loss.item():.5f}",
        f"wav_max={wav_pred.max().item():.5f}",
        f"wav_min={wav_pred.min().item():.5f}",
    )
    if vocoder_loss < 0:
        break

1 total_loss=10.79928 spectro_loss=1.39298 vocoder_loss=9.40630 wav_max=0.43339 wav_min=-0.32562
2 total_loss=6.84275 spectro_loss=1.44886 vocoder_loss=5.39389 wav_max=0.28436 wav_min=-0.28388
3 total_loss=11.08834 spectro_loss=1.36445 vocoder_loss=9.72389 wav_max=0.31684 wav_min=-0.37416
4 total_loss=18.88815 spectro_loss=1.32458 vocoder_loss=17.56357 wav_max=0.10062 wav_min=-0.53207
5 total_loss=8.60340 spectro_loss=1.41401 vocoder_loss=7.18939 wav_max=0.17414 wav_min=-0.43602
6 total_loss=6.69117 spectro_loss=1.44674 vocoder_loss=5.24442 wav_max=0.21036 wav_min=-0.15964
7 total_loss=9.98747 spectro_loss=1.44637 vocoder_loss=8.54110 wav_max=0.26169 wav_min=-0.20448
8 total_loss=4.66428 spectro_loss=1.44029 vocoder_loss=3.22400 wav_max=0.16680 wav_min=-0.17627
9 total_loss=4.28400 spectro_loss=1.36050 vocoder_loss=2.92350 wav_max=0.17912 wav_min=-0.08440
10 total_loss=4.17743 spectro_loss=1.34334 vocoder_loss=2.83409 wav_max=0.15508 wav_min=-0.31343
11 total_loss=6.94396 spectro_loss=

KeyboardInterrupt: 

In [6]:
trainer.checkpoint = 300001
trainer.save()

Saving checkpoint 300001
Saved


In [52]:
wav_pred = wav_pred.to("cpu")
print(wav_pred.shape)
wav = dataset[0][1]
print(wav.shape)

torch.Size([3, 25600])
torch.Size([3, 25600])


In [127]:
y = wav
print(y.shape, y.dtype)
# y_hat = wav_pred_temp.transpose(1, 2).unsqueeze(-1).to("cpu")
# y_hat = wav_pred_temp.transpose(1, 2).unsqueeze(-1).to("cpu")
y_hat = wav_pred.to("cpu")
print(y_hat.shape, y_hat.dtype)

torch.Size([3, 25600]) torch.float32
torch.Size([3, 25600]) torch.float32


In [94]:
y.max()

tensor(0.6752)

In [57]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(input, target, loss)

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
loss = F.cross_entropy(input, target)
print(loss)

tensor([[-1.1720, -0.6714,  0.3231, -0.4771,  1.0545],
        [ 0.1122, -0.7883,  0.2384,  1.8352,  1.2782],
        [ 2.0953,  1.7618,  0.0090, -0.8780, -0.4156]], requires_grad=True) tensor([3, 0, 4]) tensor(2.6120, grad_fn=<NllLossBackward0>)
tensor(2.0240, grad_fn=<DivBackward1>)
